In [15]:
import pandas as pd
import numpy as np

rxn = "borylation"
rxn = "dioxirane"
if rxn == "dioxirane":
    df     = pd.read_csv('../active_learning/regression/perf_clean_run.csv')
    max_   = 135
    rxn_lm = 50
elif rxn == "borylation":
    df     = pd.read_csv('../active_learning/regression/perf_clean_run_borylation_db0-01_a1.csv')
    max_   = 60
    rxn_lm = 22

In [16]:
def get_avg_vs_random(af, df=df, output="mean", max_=135):
    df_ = df[[af, 'random']]
    diff = []
    for i in range(len(df_)):
        if df_[af][i] == max_ and df_['random'][i] == max_: # both are not reaching the accuracy any points
            diff.append(np.nan)
        else:
            diff.append(df_['random'][i] - df_[af][i]) # one of them is reaching the accuracy
    if output == "median":
        return np.median(diff)
    elif output == "mean":
        return np.mean(diff)
    elif output == "diff":
        return diff

In [17]:
df_diff = pd.DataFrame()
for col in df.columns:
    if 'acqf' in col:
        df_diff[col] = get_avg_vs_random(col, df, output="diff", max_=max_)
df_diff["SMILES"] = df["Unnamed: 0"]

In [18]:
df_results_acqf = pd.DataFrame()
non_zero_counts = rxn_lm - df_diff.isna().sum()
df_results_acqf["AF"] = df_diff.columns
df_results_acqf['num_target_acc']            = non_zero_counts.values
df_results_acqf['mean_vs_rd']                = list(df_diff.mean())
df_results_acqf['median_vs_rd']              = list(df_diff.median())

df_results_acqf
#df_results_acqf.to_csv(f"acqf_results_{rxn}.csv")

,AF,num_target_acc,mean_vs_rd,median_vs_rd
0,acqf_1-db100-a1,28,40.678571,29.5
1,acqf_1-db10-a1,28,39.821429,29.0
2,acqf_1-db1-a1,28,42.285714,29.5
3,acqf_1-db0-1-a1,29,49.689655,37.0
4,acqf_1-db0-01-a1,27,49.925926,43.0
5,acqf_1-db0-001-a1,26,45.846154,38.5
6,acqf_1-db1-a2,30,46.666667,29.0
7,acqf_1-db1-a1-5,28,42.285714,29.5
8,acqf_1-db1-a0-5,27,42.037037,30.0
9,acqf_1-db1-a0,27,40.259259,30.0


In [19]:
df = df[df['random'] != max_]
#df.drop(columns=['Unnamed: 0'], inplace=True)
df_copy = df.copy() 
for col in df_copy.columns:
    if 'acqf' in col:
        df_copy[col] = df[col] - df['random']


,Unnamed: 0,acqf_1-db100-a1,acqf_1-db10-a1,acqf_1-db1-a1,acqf_1-db0-1-a1,acqf_1-db0-01-a1,acqf_1-db0-001-a1,acqf_1-db1-a2,acqf_1-db1-a1-5,acqf_1-db1-a0-5,...,acqf_3,acqf_4,acqf_4-1,acqf_5,acqf_6,acqf_7,acqf_8,acqf_9,random,SMILES
0,0,-15,-15,-16,-21,-27,-27,-16,-16,-17,...,-12,-19,-19,-22,-27,-24,11,-26,37,Cc1ccc(S(=O)(=O)OC2[C@H]3C[C@@H]4C[C@@H](C[C@H...
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,CC(C)(C)C1c2ccccc2-c2ccccc21
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,c1ccc(C2c3ccccc3-c3ccccc32)cc1
7,7,-36,-51,-34,-62,-68,-68,-34,-34,-34,...,-25,-57,-58,-70,-70,-69,-29,-63,83,CC(=O)O[C@H]1CC[C@@]2(C)[C@@H](CC(=O)[C@@H]3[C...
8,8,-6,-6,-5,-15,-15,-15,-5,-6,-6,...,-8,36,47,-16,-16,-16,85,-11,28,CC(=O)O[C@H]([C@@H](C)[C@H]1CC[C@H]2[C@@H]3COC...
10,10,-35,-42,-33,-46,-52,-53,-33,-33,-41,...,-23,-21,-19,-46,-52,-46,-17,-53,67,CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC[C@H]4...
12,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,CC(C)c1ccc2c(S(=O)(=O)Cl)cn(S(=O)(=O)c3ccccc3)...
18,18,-44,-44,-49,-64,-68,-68,-51,-49,-55,...,4,-17,-16,-67,-69,-66,3,-56,82,CC(=O)O[C@H]1CC[C@@]2(C)[C@@H](CC[C@@H]3[C@@H]...
20,20,-18,-19,-20,-30,-31,-32,-21,-21,-20,...,2,-25,-19,-32,-27,-24,39,-24,45,CC(C)[C@H]1CC[C@@]2(C)[C@@H]1CC[C@]1(C)[C@@H]2...
22,22,-61,-61,-61,-60,-60,-55,-61,-61,-60,...,33,-46,-54,-60,-51,-51,49,-39,72,CC(=O)O[C@H]1CC[C@@]2(C)[C@@H](CC[C@@H]3[C@@H]...


In [22]:
random_data = df.random.mean()
for col in df_copy.columns:
    if 'acqf' in col:
        print(col, round(100*df_copy[col].mean()/random_data,1), "% of random")

acqf_1-db100-a1 -42.1 % of random
acqf_1-db10-a1 -38.4 % of random
acqf_1-db1-a1 -42.9 % of random
acqf_1-db0-1-a1 -61.2 % of random
acqf_1-db0-01-a1 -67.5 % of random
acqf_1-db0-001-a1 -67.4 % of random
acqf_1-db1-a2 -44.2 % of random
acqf_1-db1-a1-5 -42.9 % of random
acqf_1-db1-a0-5 -50.3 % of random
acqf_1-db1-a0 -45.0 % of random
acqf_10 -23.0 % of random
acqf_2 1.4 % of random
acqf_2-1 -40.4 % of random
acqf_3 -12.6 % of random
acqf_4 -36.8 % of random
acqf_4-1 -40.1 % of random
acqf_5 -70.2 % of random
acqf_6 -50.3 % of random
acqf_7 -48.9 % of random
acqf_8 7.6 % of random
acqf_9 -55.6 % of random
